# Testing hypotheses

- This module investigates the use of data science and statistics
to answer questions about the process that generated data we observe.

- Key concepts in this module:

    * Formulation of a question: *hypothesis test*
    * Evaluating the evidence for or against the question: *test statistic*
    * What are the chances: *permutation distribution*


## Lady Tasting Tea


- This is a famous example of R. A. Fisher.

- A lady declares that by tasting a cup of tea made with milk 
she can discriminate whether
the milk or the tea infusion was first added to the cup.

<img src="https://dl.dropboxusercontent.com/u/2785709/brainder/2015/tastingtea/tea_cups.png">

### Experiment

- She is asked to taste eight cups, four of each type.

- Cups are presented in a random order.

- She correctly identifies the type of 6 out of 8 cups.

- Do you think she can really tell the difference?


## A representation of the experiment

- Let her choices either `T` or `M`. She must make 8 choices, 4 of which will be `T` and
4 of which will be `M`.

- The true types of the tea are also `T` or `M`.

- Here is an example of how we might describe the experiment

In [ ]:
lady = c('M', 'T', 'M', 'T', 'T', 'T', 'M', 'M')
truth = c('M', 'M', 'M', 'T', 'T', 'T', 'T', 'M')
number_correct = sum(lady == truth)
number_correct

## A mental model

- The variables `lady` and `truth` are just two outcomes for the experiment
in which the lady correctly identifies exactly 6 of the cups of tea.

- Is this unusual? How many other outcomes for her choices are there?

- How many other outcomes for her choices are there where she correctly identifies 6 or more?

### Permutations

- A *permutation* of a list is a reordering of the list.

- The set of all possible outcomes for `lady` is the set of all
reorderings of the list `lady`.

In [ ]:
library(iterpc)
original_list = c(1:3)
original_list

In [ ]:
I = iterpc(3, labels=original_list, ordered=TRUE)
getall(I)

### Let us count the ways...

In [ ]:
number_outcomes = 0
exactly_six = 0
six_or_more = 0

library(foreach)
I = iterpc(8, labels=lady, ordered=TRUE)
permutations_lady = iter_wrapper(I)
results = (foreach(lady_permuted=permutations_lady, .combine=rbind) 
              %do% { c(1, sum(lady_permuted == truth) == 6, sum(lady_permuted == truth) >= 6)})

totals = apply(results, 2, sum)

In [ ]:
number_outcomes = totals[1]
exactly_six = totals[2]
six_or_more = totals[3]

### What if there was no difference between the cups of tea?

- If the lady could really tell no difference between the two,
then any one of 40320 possible outcomes for her choices should
reasonably be considered equally likely.

- There were 9792 such outcomes in which she would have 
correctly identified 6 or more.

- As the choices are equally likely, the chances that she would
correctly identify 6 or more are
$$
\frac{9792}{40320} \approx 24\%.
$$

- Not that rare an occurence. The chances she would correctly identify exactly 6 is about 23%!

#### Food for thought: why didn't we also permute the `truth`?

In [ ]:
c(six_or_more / number_outcomes, exactly_six / number_outcomes)

## More data?

- The percentage of 75% seemed pretty impressive until we computed the chances
we would see such an impressive rate.

- What if she had correctly identified 60 out of 80 cups of tea?

- In this case, the number of reorderings is huge, about $10^{118}$!

- Our mental model is still valid: if the lady actually has no ability
to tell the difference between the two types of teas, then any of these
orderings is equally likely, whatever `truth` is.

- We can get a sense of how impressive this is by choosing several reorderings
at random and computing the number of matches.

In [ ]:
lady = c(rep('T', 40), rep('M', 40))
truth = c(rep('T', 40), rep('M', 40))
lady[1:10]

In [ ]:
more_than_60 = function(number_permutations) {
    exactly_sixty = 0
    sixty_or_more = 0
    matches = numeric(number_permutations)
    for (i in 1:number_permutations) {
        lady_reordering = sample(lady, length(lady), replace=FALSE)
        number_match = sum(lady_reordering == truth)
        exactly_sixty = exactly_sixty + (number_match == 60)
        sixty_or_more = sixty_or_more + (number_match >= 60)
        matches[i] = number_match
    }    
    return(list(matches=matches, exactly_sixty=exactly_sixty, sixty_or_more=sixty_or_more))
}

results = more_than_60(50000)
c(results$exactly_sixty, results$sixty_or_more)

### Maybe she does know what she's doing...

- We would really have been impressed by a 75% rate if she had tasted 80 cups of tea!

- We sometimes (though not always) saw 1 reorderings out of 50000 with a success rate of 75%.

- The probability 1/50000 might not be a great estimate of how likely she
would be to achieve a success rate of 75% or higher if she really could not
distinguish between the cups of tea.

- BUT, it certainly gives strong evidence that our mental model may be wrong...

### A visual summary

- Instead of just computing the chances above, we could record the number
of matches for each reordering of `lady` and produce a histogram.

In [ ]:
hist(results$matches, breaks=20, xlab='Matches', ylab='Count', main='')

## The null hypothesis

- Our mental model above represents how we might model the experiment
under the assumption that the lady really cannot distinguish
between different the different types.

- In statistics, we call this the *null hypothesis*.

- The permutations above represented different outcomes for our experiment.

- Under our null hypothesis, each of these outcomes was equally likely. This allowed us to compute the chances that the lady would have such a success rate of 75% or higher if she really could not tell the difference (i.e. *assuming the null hypothesis was correct.*) 

- For 6/8, the chances were about 25%, we were not very impressed. If she had achieved 60/80, we really would have been impressed. In other words, *observing something rare under the null hypothesis is evidence against the null hypothesis.*

- This is the rationale in hypothesis tests.

# A second scenario

- [A/B testing](http://getdatadriven.com/ab-significance-test) is a tool used by companies to determine the effectiveness of different strategies to attract customers.

- Here is an [example](http://getdatadriven.com/ab-significance-test) found with an easy web search.

- A company tries two strategies: 
     * 1000 customers directed to page A, with 90 successful outcomes (i.e. clickthroughs).
     * 1000 customers directed to page B, with 120 successful outcomes (i.e. clickthroughs).

- Is there a difference between the strategies?



In [ ]:
strategy_A = c(rep(0,910), rep(1,90))
strategy_B = c(rep(0,880), rep(1,120))

## Mental model

- Suppose that the 1000 customers in each arm are randomly sampled from some population. (*This assumption might not be true in practice! It is notoriously hard to obtain a random sample from a population...*)

- Suppose there is no difference between the two strategies. (*This is the null hypothesis in this example.*)

- Then, if we make one big data set of size 2000 with 210 successes and then randomly assign 1000 to `A` and the rest to `B` we will have outcomes that were as likely as the observed outcome.

- What summary should we use? A reasonable summary would seem to be the difference
between the successes between arms `A` and `B`.

In [ ]:
pooled = c(strategy_A, strategy_B)
number_trial = 10000
difference = numeric(number_trial)

for (i in 1:number_trial) {
    pooled = sample(pooled, length(pooled), replace=FALSE)
    outcome_A = pooled[1:1000] # first 1000 entries are A
    outcome_B = pooled[1001:2000] # the rest are B
    difference[i] = sum(outcome_A) - sum(outcome_B)
}

hist(difference, breaks=60);

### What are the chances?

We observed a difference with absolute value |120-90| = 30.


Out of our 5000 different reorderings, how often was the difference in the number of successes
greater than or equal to 30 (in absolute value)?

In [ ]:
mean(abs(difference) >= 30)

This is pretty unlikely. Maybe our null hypothesis is not true...

# A different method

- If we knew the relative success in each arm, we might
be able evaluate the chances above in a different way.

- For instance, suppose the true success rate in arm A was 10% in arm B it was 11%.

- In arm A the number of click-throughs can be described by tossing 1000 coins each with a 10% chance
of heads.

- In arm B the number of click-throughs can be described by tossing 1000 coins each with a 11% chance
of heads.


In [ ]:
trialA = c(1, rep(0, 9))
trialB = c(rep(1, 11), rep(0, 89))
sample_box = function(box, ndraw) {
    return(function() {sum(sample(box, ndraw, replace=TRUE))})
}
armA = sample_box(trialA, 1000)
armB = sample_box(trialB, 1000)
armAB_difference = function() {abs(armA() - armB())}

In [ ]:
sample_differences = numeric(50000)
for (i in 1:50000) {
    sample_differences[i] = armAB_difference()
}
hist(sample_differences, breaks=30)

# What are the chances?

- We see that if this were the true model that generated the data
there would be about a 8% chance of seeing an absolute difference of more than 30.

- In this example, the absolute difference is a **test statistic**.

- With models for arm A and arm B we are able to create a **sampling distribution** for the test statistic.

- Using the sampling distribution we can evaluate the chances of seeing such a large number of differences.

In [ ]:
mean(sample_differences >= 30)

# What is the right model?

- In our hypothetical example, we assumed that arm A had a success rate of 10%, and arm B had a success
rate of 11%. 

- In practice, we will not know this. 

- If we had to estimate a success rate in arm A our best guess would be 9%, and 12% in arm B.

- What if we had to estimate a success rate **if we believe the two arms have the same success rate**?

- We would probably estimate it to be 10.5%, the average of the two success rates.

In [ ]:
pooled_data = c(rep(1, 210), rep(0, 1790))
armA = sample_box(pooled_data, 1000)
armB = sample_box(pooled_data, 1000)
armAB_difference = function() { return(abs(armA() - armB())) }

In [ ]:
sample_differences = numeric(50000)
for (i in 1:50000) {
    sample_differences[i] = armAB_difference()
}
hist(sample_differences, breaks=30)

In [ ]:
mean(sample_differences >= 30)

## Mental model

- We saw that we get a very similar result for computing the chances
this way as with the permutation method.

- We came up with a way to generate data that matched
our observed data **assuming the success probabilities in the two arms are the same.**

- This assumption made us propose using each arm having 10.5% success as the *correct model*.

- Statisticians call this the model for the **null distribution**: what the typical differences
look like if both success probabilities were the same.

- Note that the typical difference also depends on the success rate, that's why we used 10.5%.

- Suppose we used a success rate of 40%.

In [ ]:
pooled_data = c(rep(1, 400), rep(0, 600))
armA = sample_box(pooled_data, 1000)
armB = sample_box(pooled_data, 1000)
armAB_difference = function() { return(abs(armA() - armB())) }
sample_differences = numeric(50000)
for (i in 1:50000) {
    sample_differences[i] = armAB_difference()
}
hist(sample_differences, breaks=30)
